In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd

In [22]:
class webcrawling_soildata():
    def __init__(self,driver, columns):
        self.df = pd.DataFrame(columns=columns)
        self.df_villages = pd.DataFrame(columns=['District','Block','Village','Farmer'])
        self.driver = driver
        
    def click_on_links(self,num):
        print(num)
        self.driver.find_element_by_css_selector("a[tabindex='"+str(num+1)+"']").click()
        time.sleep(5)
    
    def clean_scraped_data(self,div_data):
        item_remove = ['State Department of Agriculture(A.P.)','Farm Size']
        m = [x.text for x in div_data if x.text not in item_remove]
        m = m[17:]
        n = m[1::2]
        GPS =  m[-1].split(':')[1].split('Latitude')[1].split('Longitude')
        n.extend([x.strip() for x in GPS])
        return n

    def convert_webelement_to_text_list(self,data):
        return [x.text for x in data]
    
    def goto_previous_report(self):
        self.driver.find_element_by_xpath("//table[@title ='Go back to the parent report']").click()
        time.sleep(2)
    
    def goto_next_page(self):
        self.driver.find_element_by_xpath("//table[@title ='Next Page']").click()
        time.sleep(2)

    def check_page_loaded(self,page):
        delay = 3 # seconds
        try:
            myElem = WebDriverWait(self.driver, delay).until(EC.presence_of_element_located((By.ID, 'ReportViewer1_ctl09')))
            return True
        except TimeoutException:
            print("{} Loading took too much time!".format(page))
            return False
    
    def scrape_soil(self,data):
        page ='soil'
        if self.check_page_loaded(page):
            table = driver.find_element_by_css_selector("table[cols ='3']")
            div = table.find_elements_by_xpath("//tr//div[@style='word-wrap:break-word;white-space:pre-wrap;font-size:7.5pt;']")
            soil_elements = driver.find_element_by_css_selector("table[cols='6']")
            units = soil_elements.find_elements_by_xpath("//td[@style='min-width: 21.71mm;WIDTH:22.41mm;']")
            ratings_values = soil_elements.find_elements_by_xpath("//td[@style='min-width: 24.44mm;WIDTH:25.85mm;']")
            test_values= soil_elements.find_elements_by_xpath("//td[@style='min-width: 21.71mm;WIDTH:22.41mm;']//preceding-sibling::td[1]")
            units_data = convert_webelement_to_text_list(units)
            soil_data = convert_webelement_to_text_list(test_values)
            rating_data = convert_webelement_to_text_list(ratings_values)
            soil =[]
            for item in list(zip(soil_data,units_data,rating_data)):
                soil.append({'Test Value':item[0],'Unit':item[1],'Rating':item[2]})
            data.extend(clean_scraped_data(div))
            data.extend(soil)
            series = pd.Series(data, index=columns)
            self.df = df.append(series,ignore_index=True)
            self.goto_previous_report()
        
    def scrape_village_level_data(self):
        table = self.driver.find_element_by_xpath("//table[@style='border-collapse:collapse;']")
        spans = table.find_elements_by_css_selector("tr[valign='top']")
        villages =[span.text for span in spans]
        sub='village'
        village_name=''
        district_name=villages[2].split(':')[-1].strip()
        block_name = villages[3].split(':')[-1].strip()
        data= [x for x in villages[4:] if 'sample' not in x.lower()]
        for value in data:
            if sub.lower() in value.lower():
                village_name = value.split(':')[-1].strip()
                print(village_name)
                continue
            self.df_village = df_village.append(pd.Series([district_name,block_name,village_name,value.split('\n')[0]],index=['District','Block','Village','Farmer']),ignore_index=True)
 

    def scrape_farmer(self,district,block):
        if self.check_page_loaded('farmer'):
            no_of_farmer_pages = self.driver.find_element_by_css_selector("span[style = 'font-family:Verdana;font-size:8pt;white-space:nowrap;']")
            for num in range(int(no_of_farmer_pages.text.split()[0])):
                farmers = self.convert_webelement_to_text_list(driver.find_elements_by_xpath("//a[contains(@style, 'cursor:pointer;')]"))
                self.scrape_village_level_data() 
                for k in range(len(farmers)):
                    data = []
                    data.extend([district,block,farmers[k]])        
                    self.click_on_links(k)
                    self.scrape_soil(data)
                self.goto_next_page()
            self.goto_previous_report()
        
    def scrape_block(self,district):
        if self.check_page_loaded('block'):
            no_of_block_pages = self.driver.find_element_by_xpath("//span[@style = 'font-family:Verdana;font-size:8pt;white-space:nowrap;']")
            for num in range(int(no_of_block_pages.text.split()[0])):
                blocks = self.convert_webelement_to_text_list(driver.find_elements_by_xpath("//a[contains(@style, 'cursor:pointer;')]"))
                for j in range(len(blocks)):
                    block = blocks[j]
                    self.click_on_links(j)
                    self.scrape_farmer(district,block)
                    break
                break
                self.goto_next_page()
            self.goto_previous_report()
            
    def scrape_district(self):
        if self.check_page_loaded('district'):
            no_of_district_pages = self.driver.find_element_by_xpath("//span[@style = 'font-family:Verdana;font-size:8pt;white-space:nowrap;']")
            for num in range(int(no_of_district_pages.text.split()[0])):
                districts = self.convert_webelement_to_text_list(driver.find_elements_by_xpath("//a[contains(@style, 'cursor:pointer;')]"))
                for i in range(len(districts)):
                    district = districts[i]
                    self.click_on_links(i)
                    #print('distr{}'.format(i))
                    self.scrape_block(district)
                    break
                break
                self.goto_next_page()
            self.goto_previous_report()     
            
    def scrape_website(self):
        self.driver.find_element_by_xpath("//select[@id='CycleId']/option[@value='-1']").click()
        self.driver.find_element_by_id('confirmLink').click()
        iframe = self.driver.find_elements_by_tag_name('iframe')[0]
        self.driver.switch_to.frame(iframe)
        time.sleep(5)
        if self.check_page_loaded('website'):
            self.driver.find_element_by_css_selector("a[tabindex ='2']").click()
            time.sleep(5)
            self.scrape_district()

In [24]:
chrome_options = webdriver.ChromeOptions()
chrome_path = r'C:\Users\Dell\Downloads\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=chrome_path)


C:\Users\Dell\Anaconda3\envs\data-x\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
url = "https://soilhealth.dac.gov.in/PublicReports/StateWiseSampleEnteredFarmersTestedSHCPrintedBlockwise"
driver.get(url)
columns = ['District','Block','Farmer','Soil Sample Number','Date','SurveyNo.','Khasra No./ Dag No.','Irrigated','Latitude','Longitude','pH','EC1','EC2','EC3','EC4','Organice Carbon(OC)','Nitrogen(N)','Phosphorus(P)','Potassium(K)','Sulphur(S)','Zinc(Zn)','Boron(B)','Iron(Fe)','Manganese(Mn)','Copper(Cu)']
crawl_data = webcrawling_soildata(driver,columns)

In [32]:
crawl_data.scrape_website()

website Loading took too much time!


In [ ]:
crawl_data.df

In [ ]:
crawl_data.df_village

In [ ]:
craw_data.df.to_csv('soil_data.csv')
craw_data.df_village.to_csv('village_data.csv')

In [ ]:
def scrape_soil(data):
    page ='soil'
    if check_page_loaded(page):
        table = driver.find_element_by_css_selector("table[cols ='3']")
        div = table.find_elements_by_xpath("//tr//div[@style='word-wrap:break-word;white-space:pre-wrap;font-size:7.5pt;']")
        soil_elements = driver.find_element_by_css_selector("table[cols='6']")
        units = soil_elements.find_elements_by_xpath("//td[@style='min-width: 21.71mm;WIDTH:22.41mm;']")
        ratings_values = soil_elements.find_elements_by_xpath("//td[@style='min-width: 24.44mm;WIDTH:25.85mm;']")
        test_values= soil_elements.find_elements_by_xpath("//td[@style='min-width: 21.71mm;WIDTH:22.41mm;']//preceding-sibling::td[1]")
        units_data = convert_webelement_to_text_list(units)
        soil_data = convert_webelement_to_text_list(test_values)
        rating_data = convert_webelement_to_text_list(ratings_values)
        soil =[]
        for item in list(zip(soil_data,units_data,rating_data)):
            soil.append({'Test Value':item[0],'Unit':item[1],'Rating':item[2]})
        data.extend(clean_scraped_data(div))
        data.extend(soil)
        series = pd.Series(data, index=columns)
        global df
        df = df.append(series,ignore_index=True)
        goto_previous_report()
        

In [ ]:
def scrape_village_level_data():
    table = driver.find_element_by_xpath("//table[@style='border-collapse:collapse;']")
    spans = table.find_elements_by_css_selector("tr[valign='top']")
    villages =[span.text for span in spans]
    sub='village'
    village_name=''
    district_name=villages[2].split(':')[-1].strip()
    block_name = villages[3].split(':')[-1].strip()
    data= [x for x in villages[4:] if 'sample' not in x.lower()]
    for value in data:
        if sub.lower() in value.lower():
            village_name = value.split(':')[-1].strip()
            print(village_name)
            continue
        global df_village
        df_village = df_village.append(pd.Series([district_name,block_name,village_name,value.split('\n')[0]],index=['District','Block','Village','Farmer']),ignore_index=True)

In [ ]:
def scrape_farmer(district,block):
    page='farmer'
    if check_page_loaded(page):
        no_of_farmer_pages = driver.find_element_by_css_selector("span[style = 'font-family:Verdana;font-size:8pt;white-space:nowrap;']")
        for num in range(int(no_of_farmer_pages.text.split()[0])):
            farmers = convert_webelement_to_text_list(driver.find_elements_by_xpath("//a[contains(@style, 'cursor:pointer;')]"))
            scrape_village_level_data() 
            for k in range(len(farmers)):
                print('farmer{}'.format(k))
                data = []
                data.extend([district,block,farmers[k]])        
                click_on_links(k)
                scrape_soil(data)
            goto_next_page()
        goto_previous_report()

In [ ]:
def scrape_block(district):
    page ='block'
    if check_page_loaded(page):
        no_of_block_pages = driver.find_element_by_xpath("//span[@style = 'font-family:Verdana;font-size:8pt;white-space:nowrap;']")
        for num in range(int(no_of_block_pages.text.split()[0])):
            blocks = convert_webelement_to_text_list(driver.find_elements_by_xpath("//a[contains(@style, 'cursor:pointer;')]"))
            for j in range(len(blocks)):
                block = blocks[j]
                click_on_links(j)
                scrape_farmer(district,block)
                break
            break
            goto_next_page()
        goto_previous_report()

In [ ]:
def scrape_district():
    page='district'
    if check_page_loaded(page):
        no_of_district_pages = driver.find_element_by_xpath("//span[@style = 'font-family:Verdana;font-size:8pt;white-space:nowrap;']")
        print(no_of_district_pages.text)
        for num in range(int(no_of_district_pages.text.split()[0])):
            districts = convert_webelement_to_text_list(driver.find_elements_by_xpath("//a[contains(@style, 'cursor:pointer;')]"))
            for i in range(len(districts)):
                district = districts[i]
                click_on_links(i)
                #print('distr{}'.format(i))
                scrape_block(district)
                break
            break
            goto_next_page()
        goto_previous_report()

In [ ]:
def scrape_website():
    page = 'website'
    driver.find_element_by_xpath("//select[@id='CycleId']/option[@value='-1']").click()
    driver.find_element_by_id('confirmLink').click()
    time.sleep(2)
    iframe = driver.find_elements_by_tag_name('iframe')[0]
    driver.switch_to.frame(iframe)
    time.sleep(2)
    if check_page_loaded(page):
        driver.find_element_by_css_selector("a[tabindex ='2']").click()
        time.sleep(5)
        scrape_district()

In [2]:
scrape_website()

NameError: name 'scrape_website' is not defined

In [3]:
from bs4 import BeautifulSoup

In [4]:
text = """1|#||4|170|updatePanel|ReportViewer1_DocMap|<div id="ReportViewer1_ctl08" style="display:none;">
	<input type="hidden" name="ReportViewer1$ctl08$ClientClickedId" id="ReportViewer1_ctl08_ClientClickedId" />
</div>|7129|updatePanel|ReportViewer1_ctl09_ReportArea|<div NewContentType="Microsoft.Reporting.WebFormsClient.ReportAreaContent.ReportPage" ForNonReportContentArea="false" id="ReportViewer1_ctl09_VisibilityState" style="visibility:none;">
	<input type="hidden" name="ReportViewer1$ctl09$VisibilityState$ctl00" value="ReportPage" />
</div><input type="hidden" name="ReportViewer1$ctl09$ScrollPosition" id="ReportViewer1_ctl09_ScrollPosition" /><span id="ReportViewer1_ctl09_Reserved_AsyncLoadTarget"></span><div id="ReportViewer1_ctl09_ReportControl" style="display:none;">
	<span></span><input type="hidden" name="ReportViewer1$ctl09$ReportControl$ctl02" id="ReportViewer1_ctl09_ReportControl_ctl02" /><input type="hidden" name="ReportViewer1$ctl09$ReportControl$ctl03" id="ReportViewer1_ctl09_ReportControl_ctl03" /><input type="hidden" name="ReportViewer1$ctl09$ReportControl$ctl04" id="ReportViewer1_ctl09_ReportControl_ctl04" value="100" /><div style="display:none;">
		<DIV dir="LTR" style="WIDTH:100%;direction:ltr" ID="Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv"><TABLE CELLSPACING="0" CELLPADDING="0"><TR><TD ID="Pcdf772033c1641f89e29b922cc88cc0b_2_oReportCell"><TABLE CELLSPACING="0" CELLPADDING="0"><TR><TD class="A181faca2a9d242078eee7ca8b621ca73111xBc"><TABLE CELLSPACING="0" CELLPADDING="0" BORDER="0" class="A181faca2a9d242078eee7ca8b621ca73111xB"><TR><TD style="vertical-align:top"><TABLE CELLSPACING="0" CELLPADDING="0" BORDER="0" COLS="2" LANG="en-US" class="Pcdf772033c1641f89e29b922cc88cc0b_2_r10"" style="WIDTH:192.6697mm"><TR><TD style="HEIGHT:3.24mm;WIDTH:68.53mm;min-width: 68.53mm;"></TD><TD style="WIDTH:124.09mm;min-width: 124.09mm;"></TD></TR><TR VALIGN="top"><TD COLSPAN="2" style="HEIGHT:6.00mm"><TABLE CELLSPACING="0" CELLPADDING="0" LANG="en-US" style=""><TR><TD style="WIDTH:192.62mm;min-width: 191.21mm;HEIGHT:4.59mm;" class="A181faca2a9d242078eee7ca8b621ca7311"><DIV style="WIDTH:191.21mm;">Progress Report for District wise Sample Registration and Test Results entry in portal as on 04-11-2019 10 pm Cycle -II</DIV></TD></TR></TABLE></TD></TR><TR><TD COLSPAN="2" style="HEIGHT:0.68mm"></TD></TR><TR VALIGN="top"><TD style="HEIGHT:2.88mm"><TABLE CELLSPACING="0" CELLPADDING="0" LANG="en-US" style=""><TR><TD style="WIDTH:68.53mm;min-width: 67.12mm;HEIGHT:1.47mm;" class="A181faca2a9d242078eee7ca8b621ca7318"><DIV style="WIDTH:67.12mm;"><DIV class="A181faca2a9d242078eee7ca8b621ca7315"><span class="A181faca2a9d242078eee7ca8b621ca7312">State:</span><span class="A181faca2a9d242078eee7ca8b621ca7313"> </span><span class="A181faca2a9d242078eee7ca8b621ca7314">Dadra And Nagar Haveli</span></DIV><DIV class="A181faca2a9d242078eee7ca8b621ca7317"><span class="A181faca2a9d242078eee7ca8b621ca7316">&nbsp;</span></DIV></DIV></TD></TR></TABLE></TD><TD></TD></TR><TR><TD COLSPAN="2" style="HEIGHT:0.71mm"></TD></TR><TR VALIGN="top"><TD COLSPAN="2" style="HEIGHT:18.00mm"><TABLE CELLSPACING="0" CELLPADDING="0" COLS="7" BORDER="0" style="border-collapse:collapse;" class="A181faca2a9d242078eee7ca8b621ca73109"><TR HEIGHT="0"><TD style="WIDTH:0px"></TD><TD style="WIDTH:13.13mm;min-width: 13.13mm"></TD><TD style="WIDTH:61.02mm;min-width: 61.02mm"></TD><TD style="WIDTH:35.62mm;min-width: 35.62mm"></TD><TD style="WIDTH:30.33mm;min-width: 30.33mm"></TD><TD style="WIDTH:29.59mm;min-width: 29.59mm"></TD><TD style="WIDTH:22.92mm;min-width: 22.92mm"></TD></TR><TR VALIGN="top"><TD style="HEIGHT:6.00mm"></TD><TD class="A181faca2a9d242078eee7ca8b621ca7330c"><DIV class="A181faca2a9d242078eee7ca8b621ca7330">Sr.No.</DIV></TD><TD class="A181faca2a9d242078eee7ca8b621ca7334cl"><DIV class="A181faca2a9d242078eee7ca8b621ca7334">District</DIV></TD><TD class="A181faca2a9d242078eee7ca8b621ca7338c"><DIV class="A181faca2a9d242078eee7ca8b621ca7338">No. of Samples Entered</DIV></TD><TD class="A181faca2a9d242078eee7ca8b621ca7342c"><DIV class="A181faca2a9d242078eee7ca8b621ca7342">No. of Farmers Entered</DIV></TD><TD class="A181faca2a9d242078eee7ca8b621ca7346c"><DIV class="A181faca2a9d242078eee7ca8b621ca7346">Samples Tested</DIV></TD><TD class="A181faca2a9d242078eee7ca8b621ca7350c"><DIV class="A181faca2a9d242078eee7ca8b621ca7350">SHC Printed</DIV></TD></TR><TR VALIGN="top"><TD style="HEIGHT:6.00mm"></TD><TD class="A181faca2a9d242078eee7ca8b621ca7355cr"><DIV class="A181faca2a9d242078eee7ca8b621ca7355">1</DIV></TD><TD class="A181faca2a9d242078eee7ca8b621ca7359cl"><DIV class="A181faca2a9d242078eee7ca8b621ca7359"><a tabindex="1" onclick="var rp=$get('ReportViewer1_ctl09_ReportControl');if(rp&&rp.control)rp.control.InvokeReportAction('Drillthrough','47iT0R0x0:0');return false;" onkeypress="if(event.keyCode == 13 || event.which == 13){var rp=$get('ReportViewer1_ctl09_ReportControl');if(rp&&rp.control)rp.control.InvokeReportAction('Drillthrough','47iT0R0x0:0');}return false;" href="" style="cursor:pointer;" class="A181faca2a9d242078eee7ca8b621ca7359a" TARGET="_top">Dadra And Nagar Haveli</a></DIV></TD><TD class="A181faca2a9d242078eee7ca8b621ca7363cr"><DIV class="A181faca2a9d242078eee7ca8b621ca7363">2,257</DIV></TD><TD class="A181faca2a9d242078eee7ca8b621ca7367cr"><DIV class="A181faca2a9d242078eee7ca8b621ca7367">13,182</DIV></TD><TD class="A181faca2a9d242078eee7ca8b621ca7371cr"><DIV class="A181faca2a9d242078eee7ca8b621ca7371">2,139</DIV></TD><TD class="A181faca2a9d242078eee7ca8b621ca7375cr"><DIV class="A181faca2a9d242078eee7ca8b621ca7375">12,456</DIV></TD></TR><TR VALIGN="top"><TD style="HEIGHT:6.00mm"></TD><TD class="A181faca2a9d242078eee7ca8b621ca7380c Pcdf772033c1641f89e29b922cc88cc0b_2_r14">&nbsp;</TD><TD class="A181faca2a9d242078eee7ca8b621ca7384c"><DIV class="A181faca2a9d242078eee7ca8b621ca7384">Total No</DIV></TD><TD class="A181faca2a9d242078eee7ca8b621ca7388cr"><DIV class="A181faca2a9d242078eee7ca8b621ca7388">2,257</DIV></TD><TD class="A181faca2a9d242078eee7ca8b621ca7392cr"><DIV class="A181faca2a9d242078eee7ca8b621ca7392">13,182</DIV></TD><TD class="A181faca2a9d242078eee7ca8b621ca7396cr"><DIV class="A181faca2a9d242078eee7ca8b621ca7396">2,139</DIV></TD><TD class="A181faca2a9d242078eee7ca8b621ca73100cr"><DIV class="A181faca2a9d242078eee7ca8b621ca73100">12,456</DIV></TD></TR></TABLE></TD></TR><TR><TD COLSPAN="2"></TD></TR></TABLE></TD></TR></TABLE></TD></TR><TR><TD class="A181faca2a9d242078eee7ca8b621ca737c"><DIV class="A181faca2a9d242078eee7ca8b621ca737"><TABLE CELLSPACING="0" CELLPADDING="0" BORDER="0" COLS="3" LANG="en-US" class="Pcdf772033c1641f89e29b922cc88cc0b_2_r10"" style="HEIGHT:10.05mm;WIDTH:192.6697mm"><TR><TD style="HEIGHT:1.76mm;WIDTH:0.00mm"></TD><TD style="WIDTH:167.62mm;min-width: 167.62mm;"></TD><TD style="WIDTH:25.00mm;min-width: 25.00mm;"></TD></TR><TR VALIGN="top"><TD style="HEIGHT:6.00mm;WIDTH:0.00mm"></TD><TD></TD><TD><TABLE CELLSPACING="0" CELLPADDING="0" LANG="en-US" style=""><TR><TD style="WIDTH:25.00mm;min-width: 23.59mm;HEIGHT:4.59mm;" class="A181faca2a9d242078eee7ca8b621ca735"><DIV style="WIDTH:23.59mm;">Page No.1</DIV></TD></TR></TABLE></TD></TR><TR><TD style="HEIGHT:2.29mm;WIDTH:0.00mm"></TD><TD COLSPAN="2"></TD></TR></TABLE></DIV></TD></TR></TABLE></TD></TR></TABLE></DIV>
	</div>
</div><div id="ReportViewer1_ctl09_NonReportContent" style="height:100%;width:100%;">

</div>|0|hiddenField|__EVENTTARGET||0|hiddenField|__EVENTARGUMENT||5784|hiddenField|__VIEWSTATE|6dRxoe4Sod9EjDgvOa9hb2+SMuJB8ClHS6TYghJKlWUc5u3K6qi6KMCb6wc8lrEG11Rm7O88NfMrlqKyDbhHJ2WKuhjpQ7ZOu0oJAt7/bVySyhPCnmr+dM+svU4yAHwjL9sKpIxC3kCXDTiaw+O9sai7JKeH90NML5Zq4+8JnC+QFjIH5+Iw/HD+T28FGql6Na5i9d+5mbdjbdrriobVKvPTyWygFPBUw44m3jR1EZQCRNR/U1HI0m/EAHMAEbT00hqNRbGaeQ7cXy/eFn0DzN71wxRWnbM352wWC+72heW16me2s9absmpYQXi+uNup5y3W7huz80jMS7kDSc0oKXrJFel/ZJ2eYmXc1I1wTF9qZGuN4AdgLhVEVK54WC4tbomr667+f5B6YmZB2GVnDkOZTZbrAnlKvvRRQNkyHquMlMk8xp9NQHtQhFOC7PTTW8tvwEZDeTQRMb4dgmfoY7vnUzuk4JGrOBtRlCa9b2S6a/0/J0Kpz94v0mvvUCAi844BeqaShpUaeN3ZcMFm/hKCAqjU3Og/QiJ7tyCnKTZCz+kfGS2IAkOP4zyrGACAFO7UbBnop01XxlcxvEEp3E/o5/TcXrJ9sA0CYqmqskRTI3o8FCQOtOhX3a2jNQDWLqr1g3R8tlv3a0xcVK3YN8SDGuFHP8zR5KHCTfKAP2LU86+19HQot5OEufA6XG0G0/QQRa7S162tKLGr7/Who+HTLHkB5VoUJK8n+fnPcSbo+PINgkISmmjkiEY+4Cn7WP1gYfx2/pe/oDy9PRa0WyypdXuNl9k6hrF8wmwH0oJZeT6vmKMS2ymfU+5S+BiduB16hnYhheIKuRbJ8vCj4Bnl2W0/lEGQIvjja/h4JUs7S4u9LaP/cmsxS69rClkm6eT3GkLUCpDBZp/IRz3cn4jFhRwFGKmHtdMn183I4HV6WHRdtxmsvEQpuMlWRnh7kSH9pMwuZf7mrloX0vtkA7L/d7RSn54EdrN/KpfJMLuybfToOmRsmeMVPE92luVEeyyaY1T7/vycXk+2D9UnA/ziefh/qW00286N6kvfLPf0ZfxaDYpE6KQb5+nRLBKrdlBcMb4h34Cd3UdgXrcIsqrZpFakTeLYzhEMIQ0fWZTW2M0P+2gIldn1gysMGM0sBZcIXfEZm4y/KzKeXyVeX1Ab/8PeHmKyx1dnCruHwH7wCtm0YPAIxjuuVoYb8olb46RnSE139rk0mFh5ER8dEV3P92V4n1m8eks7WFCvkGO53ER+z14PrZ4vw7i0xw5gmVuBTBl1e94smbbKJT/CjHS1xLPam3NyzwaBj+VS2da4ErKvLJrSjB3D3+95pchYf0Fd5cxSzOTwwLLcBkyeyz9N3uHlwjPIVQqb4qUD2ea9QET/s8Gm8L17aYapQNxf1u9GfR70IMTf7jx1GACgu3ucjg2qcut7isvJvZk0LhCZeQc/o/yKnb86yvf/HQKkBHXZKkPXjn47MYBZwpuvPy1BzMHOMvU2W07KhUILGEQI/54FzqpmuM8R+cSiAF4jzRkbUizsq+sFR4zEe0wbdOPdfHfykp962ZrLQm6N3hGMcbpLTphVTjthl+kO6/GzcEG0RPklb3U8NNUIbj6Zzm+Kcs6GJLzbGHcoE9pkugT8ttAx/gw5xmsWZnSuIrSWH03SOAPdgcXyTWVqt4sWAsBhO4KdeGVsQj+468qN6jzvRnETwM6m2aoLJCh/L9iClwx/lULjSyLU+Tp8IBd1YiMPkBMoTgPqnN+O8Ss6n2ovTt6OdzMFq+EgkNn1mQtn/IC8qizw/KoqusyDjSnJ/+AOYDGzGcJa9qsHS/WAxfR//TMX03fH9jmm0kccEmbtmgdQjAKKLIkX8p0rfGQJECA1v3QOTg4/pidPiSO4mA9WjtKEM8m06myxzbXEt1Krll/1OJ0PNZOmNKMiDKIMJq5IT9ATTcPDlhPNhYOjTQp6SN27E5EEA5oX4zLiBtB3GU/JcpVUU5E/E6pEXI0nvQMeWNP1tbSGHrwb4SbIPq1wsLvx0z6uKOl6tkqDWtkD+uxgiCsHJEJYZMA6OFjmYqGFLpyz5pptmyC591hw2ge8GKa8q525hRz7HRQxeoXXsg6K95GKAm0o0kuwAfj8xrtSj6tEoOV2PxQAUUQHhX60wkoAFL5omon6KvTF+1TLqgepoDbjyj+0N/qIvNd5zwlSD7nIXspJ6q+9Sm9FfddaIOwBwmdgG4uW0WMK5sWrK2l/wceUkLssDn4Z9pyfEv7pQ7e6FKyFFYtRxraTKB7B3QI2avmluvGcjnDBF1VpANiIYsY3CeHTVwUvAKWCR7sT7RFHbaAAS5iug9xvp4CoF3Apy5y8y8EVmpbg+8hZKl8QfAgMpos78xylMGI+HiHdc8YMKQkwNWVIBdT7v11I4eK88hhyFOY3TvLZo56UncJi0/ZAfWCz1/6GSFLIdhIzXcmZqcQ+RulNKdSwlheLHkEO7jNu6RDw0aaGy12YKTFLf7m6pBQQc5VDomPrDqxPN3aXA0uMgPQcYxTd6ghaecQh7cJiDp51SCGMx4hMvY5U3C4iGjKXTpIwAYGlRGuRyrRUuvJtn6lCNDnJ04uZeIf8grpxIaarsWa7LWaxqC0J/y55kxpWd2Udm0jUNvziBM2Q5CHtbCTkS1PO1pXq5TW7jTuWKqFVuvg2kyU1tivsoHCYHD3hSpUVyeWb761VgbQui6BJt/kn9xdnqN7ONuc1SaX6d98Rq0aXf0qKstBCK/ytkh1wETrj3DjdoEodDLdNaQO8J4VMN1Y1mYm+gnulIJU35qV59265p2I6JpFZAzNJHeJ34bk2FFc97Lj3SWzULUTvvpIabpxU7gd24Ap6twS1J9bJeD+H/+0hQY2PzM7QhQUF0JEIsznTpkIVner1+lsm3h63pUGUGnymIKtfWCR/+GcZbgiYzhe8Xua4Iq5wL/46d7VHo3KxpI7yNT9gWycylSbMVBJBIKTn5xkii/jR7bntUoC8DLkka6+RSexeDUQT+9GLPiHF5YpC2xtYSVpOsVBvbEARYHr0sfpYLmZO824o5xHIvwGF/O47Il2pjExM0XordAfDJYviWRLl5uCRSfi1/f4oOpZ5G4Uz7FJvJy8rIekReZ4AA+KZtvfM7wjuXul42kpuJMhVUNxOttroHxc/KRddXgMgzz8ZrvRkS+6oC+jStgaAbia4f+RXPoVLVtWFg6YIb1E2yEHWIwBnLYmSy9YGSPJLHfW98woj3Fqp85nTDjcI/z6m6XSX5VWmIuAULpOHUR7onFxCfZndyoTF9ukpCSsFO2jyEhmNHWT1/R3dgylhLLN0v9ZFw+4hDkJrDZqgr2dJqObHClTka6A4UiXcxuZLd0W5E2VlL+fXzYpUR9jmSHXUF+uqX3JyBvjWZ8lWY2swfFVY72SIcE0bg2IDR1JTu6gyEtHg9vrVLT9CZbiTIqyabHJjJnJeTU7O0R6U2RW5yfbq6xYE2xvkTmHRAxbst667d8K0ZzmNmBhPPW4v682jPiy+4irBzY7YoI/DtrRnLFlq/vTmeasbpxsavUDCG5OEzqLUzjN7jBbI6FEM45DaWFGc0NJ3MPNBLp+mHP7OZAshXmVWBIFHQcBE9ffr7P1RneTduBaWBwBG9vFGnyVOjSigVk3EHhHoQnSwJ6oACsCJc0LLPBnnbjq8py9wOGC/NNzYGfdtZa4k5bTNm35Xi9coYk4iz68PkfhlH3prUXMNzX06eQQ3Hain0hGTtQBxiXmRj+wIpQW34NWn69a6Nzz/GgJfw0tPACmRBa0nIyp5aUjN/axfMW2zNG4NRe4OG3ee+y15sBz2mcvcKxLay62iAM95tYJQ7FuFuP5VGbn8JNbjH2H/dsl3uavFha1Xek8FTzHjSn6eRake1USsScxqo6vXptXn/mlkUt3TIGEzWvPXnihIu4QO4eURGje8PZGHQ2+3f3HZ/S7D+Ct8Rew+ArZ0Rub9vg0pBFm3j6e9qZ+8x83Vq2648f8Vzih3/21QdOG49UHVQlr/Y6UvlT6K7esuPxTfzHEXAeDZC8u05omd8Yje/t9luVN4nkQiSyX4EsuH2gJ3DuS662LcuOOQNXh0p+dg3rm+QdXf7HOVhq7PwPNNEZNOvSIhaW6q7kq244jGJejQjBH2LvpXRWSHae9dPe5G5Ehyvm3wg99rQp76p7/3uhtux2NbIDjJoB3fzxSkxkiLkxqpoLsXFohmEToczxMM7t3cytMLWSlzeagmV+9aM4/+cb0r1W86KLhqGIha2RfsGfdLVoYXE16qJ1aQ5ZrVETRJ2sgIqGQmldSPi2FENR/Ha9Odr0VIkIzxUIgGw4KXyeD0axR8bBaQBkPXoeso9ezIRAIMrAjrbgAQ8cwzQH3VgbfpVXM01KxnPrFrMQRgQLkD0gUOfTWeMhoDG86k76B6KQu++V0iAiGhBoVfS4DYd6sdS8r3OB1cqX2kfZGzHX0zhRL9m3/GGZZQGrl64SfEl86VZQ+w2jxlPzXBfra8ohwwFAJxAPA1rSgvH5WNDwOkfpEIYueDF5U1DNRXSsghUXyDn8nwp98yjiBItBKzEZ4xAaSp31MUQX8l7DzLSQJ4+Fc4d17GTk/6UTOKA5tMIShYJwL14tGKxAcCB9UJIY+WtvMSFgY/NcHBzO+2iye1Tmr9/SywxqRYNES7i1UBmURSwvKsEGoTOJfG8K9msvBVMDQ0XOFyoYPlPabBm8hDfYA0r6qMeHhgouzdV120IlBslEcsbjGAa2qL7S+yecbHZE4oCMfGWMEkOcpJ8d+kHIk46CPFsHKjihfnnNldYUKeLMUd5kXc47RMLYo2KsXJHxrAsrAbX7p+8tANwum8Qn5EQdGyLxdDrhNAmoHCpiWgyv0D4Mg+AAnYDXhtDFOmhiWSGZqwHm8nqNvSDsMUIx8IQD3g6uXz47dzNAC+UcfIv9SiLjwMYOuU4sNRYJnt1eb+jgENKVQZGNprClhq0WCwGIA7Nu06v8pTlWnBFPBDE8q64c5cEHdY3HSSffrGVzNF3eqw6u+lkAepDkTOfponq4NZ+5JAGPbOfl/cPs6NCJUvFLp1+diYWCr7Bk+jocbDKruSSQEd1lROS6rMSKw229qn9sTxlPUVWtH5ZGur1MeTWC2OtW+B0MXbxeqv/fJJnNDHl/9A0wb/vHuJTSfJvnVQzDKZuIfI+f11JYBEx3f+mkj/hLBPB7sEI0i2IxMgleLWjpZgFL3n9ebuqcYx8WySSccZ3Xq62mm6o7WgCRoKCZboQoE+C1B+sp4jVHG38YgxbrJkN0reqUMrGbcNq7nGYXZp99qAaVhqivmqx4xFgDKjRGWH3f6JtwOB8xwkWpjBFL2cQZ32MVHBLd3TCI8NAlNjtixVo0QAZxJnH3nRrg6uIIrMyYVTAPxsmBJWNtSgimGPtqDLuf1JgwwmYBYPRZPQwtqt/Vn2LQ1sxLfl5/ruUrvvbPKNJ2g3bm9gQO3cGK/9R181ityN6IYGvfNZgfFejyi0PSzIkkbRDFfgkVD2jzxlbXdBGjRZnJO9bWIPAI/+/HWYEe4iTqhk+VLIhPXrJQz26UyC31yXqLF91AZ5UdEAbbrWKhwUFqq50Fn1xi9aCzb6FRIdIE2B8NQpKXlQa9DuVHAWygETu78HBvdoSfYug26fyzvDoDf//ocyY9gmuGXRlUYO0E5CBhLnOmcYY8BcTWLAFKxzOrkoTKGxlAbGIdgYS/TjOScvn7KcL3PxIbUNGHfgMFld0Vv71fSW6f1oyFkWfw==|8|hiddenField|__VIEWSTATEGENERATOR|878B99D9|984|hiddenField|__EVENTVALIDATION|ieU5BWefg2n+KY/oNMdUhikjS4zL4QSXduEsmOKG2+n2YFkKFq7vAzQFXk66mKN7gd5D1TEBzo7uTpgnx0T2JfY9VR33jUq3uEKL8zd9DluR2BzxITQjeKOJDzbQL6QmJ7kKRDpM0WHKY2yXPoQ/G0u11wrntn8lggstAq442gze3GsgAu0h4BYlpE/rEw6Pw+lyeRw5thDp8as1VHeJAZONJU4MOh9JDMZim4Ft0GvLVkYrF7onh3RCzRfXNZ8vEwmA9qyd2ij7ipyhnSTCH2oVcVqCTAE/NlfZ0hv0lZSdlq+nblqdjuLNMuzyTWJ8MwM9frvSyebTCvfuLKpFCNBINu8xpHjq730I3Bkya+9L3Umn9kL/bRXlcVUAw6lT+Q4DGTkuRNp+ZECoKYrJilI1u2Xovm8zQDuiSzvfOi210nxOg9MivypSocnNOmPS9cpekaz4JAiojxSgrF3qARJuUWecx98xHosZG9dihIDlfHIPdouWgSlo3X/XeEt9wATZFo6thN0atFfQwgqEn806WKHXCUs3Y/D6hl3A47sgZZnDudSdncHEXvKSF2X1k9vSVQP3MsaItclMy3dMbBIAyJCcg1Dxg3O4ddiALC7t6lPZF6sxzMok7YUYtNcONJ3Vt363UHdVOW7iPhiMguXs27Cf2WYXVtM5CmblmhRRWbE3lFTJupgzI93p+J2xx/F3O8gOnaacFhBYa4tz3SBJay9w45a+vghrPY3A9cbje+HXq0wLNaVClb2i6DR4Vom9TXY6HRVo9SrysvQlRJEDRCcOT2MOZ7BoKdOvqyuEufcSetvEO2UT/WZKXaOwtbrzIGZmVIn6VjN5iYR2mTUdgOdCBDpWs0bb15JwTX5NGyXf7X+M6Z5sa1/7e3YmmYTGok2pFYn1LRWEXNbGpPG2nK4Ur/MGlDY0M92OR/7HPfg6behzLPhYXR6hEtgXg8Ugp5RhvLAenRdXo6Tb2Q==|0|asyncPostBackControlIDs|||40|postBackControlIDs||ReportViewer1$ctl09$ReportControl$ctl00,|84|updatePanelIDs||fReportViewer1$ReportViewer,,fReportViewer1$DocMap,,fReportViewer1$ctl09$ReportArea,|0|childUpdatePanelIDs|||53|panelsToRefreshIDs||ReportViewer1$DocMap,,ReportViewer1$ctl09$ReportArea,|2|asyncPostBackTimeout||90|13|formAction||./Report.aspx|90|scriptBlock|ScriptPath|/Reserved.ReportViewerWebControl.axd?OpType=Resource&Version=11.0.3452.0&Name=ViewerScript|213|scriptBlock|ScriptPath|/ScriptResource.axd?d=D1hOhMuFPfzUyl--01QXms3pCnSjjiQB6-GwcdPEDdx8aCdLKDNQ1Mho4U9A1VuBY0YSjRR_6BS6tqQEcmbzlhrIOTIwy2qYDPAKSIGUpt5jRvIiWkbi6U8tVb5ZyB053UapJ0xBDPx7QKUGrqFURAyUAGahrDip6UxSIrLUHl81&t=ffffffffa580202a|18411|scriptStartupBlock|ScriptContentNoTags|Sys.Application.add_init(function() {
    $create(Microsoft.Reporting.WebFormsClient._ReportPage, {"ActionParamId":"ReportViewer1_ctl09_ReportControl_ctl03","ActionTypeId":"ReportViewer1_ctl09_ReportControl_ctl02","HiddenZoomLevelId":"ReportViewer1_ctl09_ReportControl_ctl04","InteractivityMode":"AlwaysAsynchronous","PrefixId":"Pcdf772033c1641f89e29b922cc88cc0b_2_","ReportCellId":"Pcdf772033c1641f89e29b922cc88cc0b_2_oReportCell","ReportDivId":"Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv","ReportPageStyles":"background-color:Transparent;border:1pt none Black;","ReportStyles":"#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca73111xBc{border:1pt none Black;background-color:Transparent;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca73111xB{border:1pt none Black;background-color:Transparent;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7311{word-wrap:break-word;white-space:pre-wrap;padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt none Black;background-color:Transparent;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:700;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:center;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7318{word-wrap:break-word;white-space:pre-wrap;padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt none Black;background-color:Transparent;font-size:0pt;unicode-bidi:normal;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7315{text-align:left;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7312{font-family:Verdana;font-size:10pt;font-weight:700;font-style:normal;text-decoration:none;color:Black;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7313{font-family:Verdana;font-size:10pt;font-weight:700;font-style:normal;text-decoration:none;color:Black;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7314{font-family:Verdana;font-size:10pt;font-weight:700;font-style:normal;text-decoration:none;color:Black;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7317{text-align:left;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7316{font-family:Verdana;font-size:10pt;font-weight:700;font-style:normal;text-decoration:none;color:Black;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca73109{border:1pt none Black;background-color:Transparent;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7330c{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:#effae2;vertical-align:top;text-align:right;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 11.02mm;WIDTH:13.13mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7330{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:700;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7334cl{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:#effae2;vertical-align:top;text-align:left;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 58.91mm;WIDTH:61.02mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7334{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:700;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:left;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7338c{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:#effae2;vertical-align:top;text-align:right;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 33.51mm;WIDTH:35.62mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7338{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:700;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7342c{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:#effae2;vertical-align:top;text-align:right;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 28.21mm;WIDTH:30.33mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7342{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:700;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7346c{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:#effae2;vertical-align:top;text-align:right;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 27.47mm;WIDTH:29.59mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7346{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:700;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7350c{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:#effae2;vertical-align:top;text-align:right;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 20.81mm;WIDTH:22.92mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7350{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:700;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7355cr{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:Transparent;vertical-align:top;text-align:right;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 11.02mm;WIDTH:13.13mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7355{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:400;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7359cl{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:Transparent;vertical-align:top;text-align:left;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 58.91mm;WIDTH:61.02mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7359{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:400;text-decoration:underline;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:left;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7359a{font-family:Verdana;font-size:8pt;font-weight:400;font-style:normal;text-decoration:underline;color:Black;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7363cr{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:Transparent;vertical-align:top;text-align:right;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 33.51mm;WIDTH:35.62mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7363{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:400;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7367cr{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:Transparent;vertical-align:top;text-align:right;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 28.21mm;WIDTH:30.33mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7367{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:400;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7371cr{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:Transparent;vertical-align:top;text-align:right;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 27.47mm;WIDTH:29.59mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7371{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:400;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7375cr{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:Transparent;vertical-align:top;text-align:right;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 20.81mm;WIDTH:22.92mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7375{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:400;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7380c{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:Transparent;vertical-align:top;text-align:left;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 11.02mm;WIDTH:13.13mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7384c{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:#effae2;vertical-align:top;text-align:center;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 58.91mm;WIDTH:61.02mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7384{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:700;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:center;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7388cr{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:#effae2;vertical-align:top;text-align:right;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 33.51mm;WIDTH:35.62mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7388{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:700;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7392cr{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:#effae2;vertical-align:top;text-align:right;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 28.21mm;WIDTH:30.33mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7392{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:700;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7396cr{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:#effae2;vertical-align:top;text-align:right;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 27.47mm;WIDTH:29.59mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca7396{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:700;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca73100cr{padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt solid #d3d3d3;background-color:#effae2;vertical-align:top;text-align:right;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;min-width: 20.81mm;WIDTH:22.92mm;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca73100{word-wrap:break-word;white-space:pre-wrap;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:700;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca737c{border:1pt none Black;background-color:Transparent;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca737{min-width: 192.67mm;min-height: 10.05mm;WIDTH:192.67mm;border:1pt none Black;background-color:Transparent;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .A181faca2a9d242078eee7ca8b621ca735{word-wrap:break-word;white-space:pre-wrap;padding-left:2pt;padding-top:2pt;padding-right:2pt;padding-bottom:2pt;border:1pt none Black;background-color:Transparent;font-style:normal;font-family:Verdana;font-size:8pt;font-weight:700;text-decoration:none;unicode-bidi:normal;color:Black;direction:ltr;layout-flow:horizontal;writing-mode:lr-tb;vertical-align:top;text-align:right;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r1{HEIGHT:100%;WIDTH:100%}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r2{HEIGHT:100%;WIDTH:100%;overflow:hidden}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r3{HEIGHT:100%}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r4{border-style:none}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r5{border-left-style:none}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r6{border-right-style:none}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r7{border-top-style:none}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r8{border-bottom-style:none}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r10{border-collapse:collapse}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r9{border-collapse:collapse;table-layout:fixed}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r11{WIDTH:100%;overflow-x:hidden}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r12{position:absolute;display:none;background-color:white;border:1px solid black;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r13{text-decoration:none;color:black;cursor:pointer;}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r14{font-size:0pt}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r15{direction:RTL;unicode-bidi:embed}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r16{margin-top:0pt;margin-bottom:0pt}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r17{HEIGHT:100%;WIDTH:100%;display:inline-table}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv .Pcdf772033c1641f89e29b922cc88cc0b_2_r18{HEIGHT:100%;display:inline-table}#Pcdf772033c1641f89e29b922cc88cc0b_2_oReportDiv  * { -webkit-box-sizing: border-box; box-sizing: border-box }","ScrollableContainerId":"ReportViewer1_ctl09","SearchHitPrefix":"Pcdf772033c1641f89e29b922cc88cc0b_2_oHit","StyleElementId":"ReportViewer1_ctl09_ReportControl_styles","ToolBarUpdate":{'CurrentPage':1,'TotalPages':1,'IsEstimatePageCount':false,'TotalPagesString':'1','SearchText':'','CanFindNext':false},"TriggerAsyncInteractivity":function(){__doPostBack('ReportViewer1$ctl09$ReportControl','');},"TriggerSyncInteractivity":function(){__doPostBack('ReportViewer1$ctl09$ReportControl$ctl00','');}}, null, null, $get("ReportViewer1_ctl09_ReportControl"));
});
|"""

In [5]:
soup = BeautifulSoup(text)

In [11]:
soup.table.td

<td id="Pcdf772033c1641f89e29b922cc88cc0b_2_oReportCell"><table cellpadding="0" cellspacing="0"><tr><td class="A181faca2a9d242078eee7ca8b621ca73111xBc"><table border="0" cellpadding="0" cellspacing="0" class="A181faca2a9d242078eee7ca8b621ca73111xB"><tr><td style="vertical-align:top"><table border="0" cellpadding="0" cellspacing="0" class="Pcdf772033c1641f89e29b922cc88cc0b_2_r10" cols="2" lang="en-US" style="WIDTH:192.6697mm"><tr><td style="HEIGHT:3.24mm;WIDTH:68.53mm;min-width: 68.53mm;"></td><td style="WIDTH:124.09mm;min-width: 124.09mm;"></td></tr><tr valign="top"><td colspan="2" style="HEIGHT:6.00mm"><table cellpadding="0" cellspacing="0" lang="en-US" style=""><tr><td class="A181faca2a9d242078eee7ca8b621ca7311" style="WIDTH:192.62mm;min-width: 191.21mm;HEIGHT:4.59mm;"><div style="WIDTH:191.21mm;">Progress Report for District wise Sample Registration and Test Results entry in portal as on 04-11-2019 10 pm Cycle -II</div></td></tr></table></td></tr><tr><td colspan="2" style="HEIGHT:0.

In [8]:
import pandas as pd

In [9]:
pd.read_html(soup.table)

TypeError: 'NoneType' object is not callable